In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/200000-jeopardy-questions/JEOPARDY_CSV.csv")

In [ ]:
df.head()

In [ ]:
print("Shape of Dataset",df.shape)

In [ ]:
df.info()

Step 2: Data Understanding and Cleaning

In [ ]:
df.isna().sum()

In [ ]:
df.isnull().sum()

In [ ]:
# Let's fill the answer with maximun number of repeating answer. Since only 2 of them is missing
df[' Answer'] = df[' Answer'].fillna(df[' Answer'].mode()[0])

In [ ]:
# drop duplicate values
df.drop_duplicates()

df.shape

No NULL and duplicate values present in dataset

# **Step 3: Visualizing the Data**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df.describe(percentiles=[.25,.5,.75,.90,.95,.98,.99])

In [ ]:
df.columns

In [ ]:
df = df.rename(columns = {"Show Number":"Show_Number"})
df = df.rename(columns = {" Air Date":"Air_Date"})
df = df.rename(columns = {" Round":"Round"})
df = df.rename(columns = {" Category":"Category"})
df = df.rename(columns = {" Value":"Value"})
df = df.rename(columns = {" Question":"Question"})
df = df.rename(columns = {" Answer":"Answer"})

In [ ]:
df['Category'].value_counts()

In [ ]:
#total categories
len(df['Category'].unique())

In [ ]:
#total unique shows
len(df['Show_Number'].unique())

In [ ]:
#total unique shows
len(df['Value'].unique())

In [ ]:
df['Value'].value_counts()

In [ ]:
count=0
for val in df['Value']:
    if "$" not in val:
        count=count+1
print(count)
print("percentage of invalid value",count/len(df))

In [ ]:
#removing invalid values

df = df[df['Value'] != "None"]
df.shape

In [ ]:
#converting value to numeric datatype
def converticToNum(x):
    #print(x)
    return int(x[1:].replace(",","").replace(" ",""))
df['Value'] = df['Value'].apply(converticToNum)

In [ ]:
df['Value'].describe(percentiles=[.25,.5,.75,.90,.95,.98,.99])

In [ ]:
#visualizing target variable

sns.distplot(df['Value'],bins=10)
plt.show()

value coulmun is skewed, creating bins to reduce feature values

In [ ]:
def binning(value):
    if value < 1000:
        return np.round(value, -2)
    elif value < 10000:
        return np.round(value, -3)
    else:
        return np.round(value, -4)

df['Value'] = df['Value'].apply(binning)


In [ ]:
df['Value'].value_counts()

In [ ]:
#removing 20000 as count is very less
df = df[df['Value'] != 20000] 

In [ ]:
df['Round'].value_counts()

In [ ]:
#percentage count
df['Round'].value_counts()/len(df)

In [ ]:
sns.countplot(df['Round'])

In [ ]:
#remove all the rounds with value Final Jeopardy and Tiebreaker as they account for 0.016% and 0.000014% of total values

In [ ]:
df = df[df['Round'] !="Tiebreaker"]
df = df[df['Round'] !="Final Jeopardy!"]

In [ ]:
df.shape

In [ ]:
sns.countplot(df['Round'])

# **Logistic Regression model**

In [ ]:
df.sample(n=5)

In [ ]:
import re
def rephrase(sent):
    sent = re.sub(r"won't", "will not", sent)
    sent = re.sub(r"can\'t", "can not", sent)
    sent = re.sub(r"\'m", " am", sent)
    sent = re.sub(r"n\'t", " not", sent)
    sent = re.sub(r"\'s", " is", sent)
    sent = re.sub(r"\'d", " would", sent)
    sent = re.sub(r"\'re", " are", sent)
    sent = re.sub(r"\'ll", " will", sent)
    sent = re.sub(r"\'t", " not", sent)
    sent = re.sub(r"\'ve", " have", sent)
    
    return sent

In [ ]:
from nltk.corpus import stopwords

stopWords = set(stopwords.words('english'))

stopWords

negative_words=["aren't","couldn't","didn't","doesn't","don't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","no","nor","not","shan't","shouldn't","wasn't","weren't","won't","wouldn't"]


In [ ]:
stopwords=[]
for word in list(stopWords):  # iterating on a copy since removing will mess things up
    if word not in negative_words:
        stopwords.append(word)


In [ ]:

preprocessed_question = []
for sentance in (df['Question'].values):
    sent = rephrase(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_question.append(sent.lower().strip())

In [ ]:
df['Preprocessed_Question'] = preprocessed_question    #create new column having name  with preprocessed data
df.drop(['Question', 'Air_Date'], axis=1, inplace=True) #delete the column


In [ ]:
df.head(5)

In [ ]:
#train_test_split
from sklearn.model_selection import train_test_split
target=df['Value']
data_train, data_test, data_y_train, data_y_test = train_test_split(df, target, test_size=0.33, stratify=target)
data_train, data_cv,data_y_train, data_y_cv = train_test_split(data_train, data_y_train, test_size=0.33, stratify=data_y_train)

In [ ]:
data_train.sample(n=5)

In [ ]:
# Question
#https://stackoverflow.com/questions/48090658/sklearn-how-to-incorporate-missing-data-when-one-hot-encoding
from sklearn.feature_extraction.text import CountVectorizer
vectorizer3 = CountVectorizer(lowercase=False, binary=True, max_features=2000)
vectorizer3.fit(project_data_train['Preprocessed_Question'].values)
#print(vectorizer3.get_feature_names())

feat_1_train = vectorizer3.transform(project_data_train['Preprocessed_Question'].values)
feat_1_cv = vectorizer3.transform(project_data_cv['Preprocessed_Question'].values)
feat_1_test = vectorizer3.transform(project_data_test['Preprocessed_Question'].values)

print("After vectorizations")
print(feat_1_train.shape, project_data_y_train.shape)
print(feat_1_cv.shape, project_data_y_cv.shape)
print(feat_1_test.shape, project_data_y_test.shape)
print("="*100)
